In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset

In [2]:
dataset = load_dataset("stanfordnlp/imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [17]:
import pandas as pd

df=pd.DataFrame(dataset['train'])
positive_comments = df[df['label'] == 1]['text'].tolist()
negative_comments = df[df['label'] == 0]['text'].tolist()

pairs = [(pos, neg) for pos in positive_comments for neg in negative_comments]

print(f"Number of pairs: {len(pairs)}")


Number of pairs: 156250000


In [18]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Загрузка токенайзера и модели
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-cased")

# Создание DataFrame с парами для удобства обработки
pairs_df = pd.DataFrame(pairs, columns=["positive", "negative"])

# Токенизация пар
def tokenize_function(example):
    return tokenizer(example['positive'], example['negative'], truncation=True, padding='max_length', max_length=128)

# Токенизация и создание датасета
tokenized_pairs = pairs_df.apply(tokenize_function, axis=1)

# Разделение на тренировочную и тестовую выборки
train_df, test_df = train_test_split(tokenized_pairs, test_size=0.1)

# Создание Dataset
class PairDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = df['label'].tolist()
        self.encodings = df['input_ids'].tolist()

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PairDataset(train_df)
test_dataset = PairDataset(test_df)

# Настройки обучения
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
)

# Обучение модели
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset             
)

trainer.train()


C:\Users\PavelAgafonov\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MemoryError: Unable to allocate 1.16 GiB for an array with shape (156250000,) and data type int64

In [3]:
I = 2
M = 2
T = 9000
lambda_=1/M
mu = 0.01  # EMA update rate
beta = 0.1  # KL regularization coefficient
eta = 0.3  # LITI update rate for the second iteration, use 0.5 if limited compute budget
batch_size = 64
learning_rate = 5e-5

In [4]:
from transformers import AutoModelForSequenceClassification

# Load pre-trained model and tokenizer
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Reward model
reward_model_name = "distilbert-base-cased"
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

C:\Users\PavelAgafonov\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\PavelAgafonov\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PavelAgafonov\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

C:\Users\PavelAgafonov\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PavelAgafonov\.cache\huggingface\hub\models--distilbert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\PavelAgafonov\miniconda3\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
def reward_fn(output, target):
    return torch.nn.functional.mse_loss(output, target)

# Create prompt dataset
def create_prompt_dataset(dataset, num_samples=100):
    prompts = []
    for i in range(num_samples):
        text = dataset['train'][i]['text']
        prompt = tokenizer.encode(text, return_tensors='pt')[:, :15]  # Take first 15 tokens
        prompts.append(prompt)
    return prompts


In [8]:
prompt_dataset = create_prompt_dataset(dataset)
data_loader = DataLoader(prompt_dataset, batch_size=batch_size, shuffle=True)

# Initialize weights
theta_sft = model.state_dict()
theta_init = theta_sft

In [12]:
# WARP Algorithm
for i in range(I):
    for m in range(M):
        theta_m = theta_init
        theta_m_ema = theta_init
        for t in range(T):
            for batch in data_loader:
                optimizer.zero_grad()
                
                # Generate completion
                inputs = batch.to('cuda')
                outputs = model.generate(inputs, max_length=30)
                
                # Compute reward
                rewards = reward_model(outputs)
                kl_div = torch.nn.functional.kl_div(outputs.log_softmax(dim=-1), inputs.log_softmax(dim=-1), reduction='batchmean')
                r_beta = reward_fn(rewards, outputs) - beta * kl_div
                
                # Update theta_m
                loss = -r_beta.mean()
                loss.backward()
                optimizer.step()
                
                # Update theta_m_ema
                for param_ema, param in zip(theta_m_ema.values(), model.parameters()):
                    param_ema.copy_(param_ema * (1 - mu) + param * mu)
        
        # Update theta_slerp
        if m == 0:
            theta_slerp = theta_m_ema
        else:
            for param_slerp, param_ema in zip(theta_slerp.values(), theta_m_ema.values()):
                param_slerp.copy_(param_slerp * (1 - lambda_) + param_ema * lambda_)
    
    # Update theta_init
    for param_init, param_slerp in zip(theta_init.values(), theta_slerp.values()):
        param_init.copy_(param_init * (1 - eta) + param_slerp * eta)

# Save the final model
model.save_pretrained("path_to_save_model")

AssertionError: Torch not compiled with CUDA enabled